In [15]:
import json
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn import preprocessing
from sklearn import cross_validation

In [2]:
FEATURES = [
    "top_oper_cli", 
    "gen_oper_cli",
    "top_proc_cli", 
    "gen_proc_cli", 
]

In [ ]:
clinic_ids = [i['id'] for i in estims_data]
#estims_data.pop(clinic_ids.index('0'))
#print sorted(clinic_ids)
#print sorted([i['id'] for i in cancer_data])

In [3]:
def extract_data(clinic, clinic_estim):
    all_proc = clinic['procedures']['diagnostics'] + clinic['procedures']['operations']
    X = []
    X+=[sum([int(i['top']) for i in clinic['procedures']['operations']])] #top_oper_cli
    X+=[len(clinic['procedures']['operations'])] #gen_oper_cli
    X+=[sum([int(i['top']) for i in all_proc])] #top_proc_cli
    X+=[len(all_proc)] #gen_proc_cli
    y=int(clinic_estim['variety'])
    return [X, y]

In [4]:
def get_X_y_from(t_data):
    clinic_ids = [i['id'] for i in t_data]
    t_cleaned_data = [extract_data(cancer_data[clinic_ids.index(i['id'])], i) for i in estims_data]
    X = [i[0] for i in t_cleaned_data]
    y = [i[1] for i in t_cleaned_data]
    #X = np.array([i[0] for i in t_cleaned_data])
    #y = np.array([i[1] for i in t_cleaned_data])
    return X,y

In [5]:
def normale(mat):
    for i in [0,1,2,3]:
        mass = [k[i] for k in mat]
        m = max(mass)
        if m>0:
            for j in mat:
                j[i] = round(j[i] / float(m), 4)

In [6]:
def process_with(X,y, info=False, short=False, return_short = False, new_coef = [], ts=0.2):
    train_X, test_X, train_y, test_y = cross_validation.train_test_split(X, y, test_size = ts, random_state = 3)
    regr = linear_model.LinearRegression(fit_intercept=False, normalize=False)
    if new_coef != []:
        print "new coefs"
        regr.coef_ = new_coef
    regr.fit(train_X, train_y)
    if info:
        print "Total: %d, train: %d, test: %d" %(len(X), len(train_X), len(test_X))
        print("Residual sum of squares: %.2f"% np.mean((regr.predict(test_X) - test_y) ** 2))
        print("Train absolute: %.2f"% np.mean(abs(regr.predict(train_X) - train_y)))
        print("Test absolute: %.2f"% np.mean(abs(regr.predict(test_X) - test_y)))
        print("Absolute to mean: %.2f%%"% (np.mean(abs(regr.predict(test_X) - test_y))/np.mean(test_y)*100))
        print('Train variance score: %.2f' % regr.score(train_X, train_y))
        print('Test variance score: %.2f' % regr.score(test_X, test_y))
    if short:
        print "Total: %d, train: %d, test: %d" %(len(X), len(train_X), len(test_X))
        print "%.3f" % np.mean(abs(regr.predict(train_X) - train_y))
        print "%.3f" % np.mean(abs(regr.predict(test_X) - test_y))
        print "%.3f" % (np.mean(abs(regr.predict(test_X) - test_y))/np.mean(test_y)*100)
        print "%.3f" % regr.score(train_X, train_y)
        print "%.3f" % regr.score(test_X, test_y)
    if return_short:
        return np.mean(abs(regr.predict(test_X) - test_y)),regr.score(test_X, test_y)
    for i in regr.coef_:
        print "%.3f" % i
    #print "%.3f" % regr.intercept_
    return regr

In [21]:
X_all, y_all = [], []

with open('./max/estims.json') as data_file: 
    estims_data = json.load(data_file)[2]['clinics']
with open('./max/меланома_все.json') as data_file: #2
    cancer_data = json.load(data_file)[0]['clinics']
X_1, y_1 = get_X_y_from(cancer_data)
X_all+=X_1
y_all+=y_1
    
with open('./max/estims.json') as data_file: 
    estims_data = json.load(data_file)[0]['clinics']
with open('./max/рак_груди_все.json') as data_file: #0
    cancer_data = json.load(data_file)[0]['clinics']
X_2, y_2 = get_X_y_from(cancer_data)
X_all+=X_2
y_all+=y_2
    
with open('./max/estims.json') as data_file: 
    estims_data = json.load(data_file)[4]['clinics']    
with open('./max/рак_простаты_все.json') as data_file: #4
    cancer_data = json.load(data_file)[0]['clinics']
clinic_ids = [i['id'] for i in estims_data]
estims_data.pop(clinic_ids.index('0'))
X_3, y_3 = get_X_y_from(cancer_data)
X_all+=X_3
y_all+=y_3

with open('./max/estims.json') as data_file: 
    estims_data = json.load(data_file)[1]['clinics']     
with open('./max/рак_шейки_матки_все.json') as data_file: #1
    cancer_data = json.load(data_file)[0]['clinics']
X_4, y_4 = get_X_y_from(cancer_data)
X_all+=X_4
y_all+=y_4
    
with open('./max/estims.json') as data_file: 
    estims_data = json.load(data_file)[3]['clinics']         
with open('./max/рак_щитовидки_все.json') as data_file: #3
    cancer_data = json.load(data_file)[0]['clinics']
X_5, y_5 = get_X_y_from(cancer_data)
X_all+=X_5
y_all+=y_5

normale(X_all)

In [ ]:
X, y = get_X_y_from(cancer_data)

In [12]:
for i in range(1,5):
    regr = process_with(X_all, y_all, short=True, ts=i/10.0)
    print

Total: 49, train: 44, test: 5
1.564
1.393
16.988
-0.090
-12.650
-1.547
-0.204
1.837
0.139

Total: 49, train: 39, test: 10
1.551
1.634
19.685
0.004
-4.523
-1.455
-0.262
1.641
0.185

Total: 49, train: 34, test: 15
1.540
1.562
19.369
0.103
-4.457
-1.182
-0.238
1.624
0.171

Total: 49, train: 29, test: 20
1.582
1.494
18.216
0.147
-3.782
-1.319
-0.292
1.609
0.205



In [22]:
regr = process_with(X_all, y_all, info=True, ts=0.1)

Total: 49, train: 44, test: 5
Residual sum of squares: 2.18
Train absolute: 1.56
Test absolute: 1.39
Absolute to mean: 16.99%
Train variance score: -0.09
Test variance score: -12.65
-3.094
-7.545
11.025
9.011


In [26]:
import warnings
warnings.filterwarnings("ignore")
for i in range(0,len(y_4)):
    print "Real: %f \t Predicted: %f" %(y_4[i], regr.predict(X_4[i]))
    print X_4[i]
    print sum(np.array(X_4[i]) * np.array([-3.094, -7.545, 11.025, 9.011]))
    print

Real: 10.000000 	 Predicted: 7.559590
[1.0, 1.0, 0.8333, 1.0]
7.5591325

Real: 8.000000 	 Predicted: 7.005449
[1.0, 0.5405, 0.8333, 0.5538]
7.0053518

Real: 9.000000 	 Predicted: 10.828023
[0.5, 0.4595, 0.8333, 0.7385]
10.8278285

Real: 10.000000 	 Predicted: 8.241414
[0.5, 0.5405, 0.6667, 0.7231]
8.2411491

Real: 9.000000 	 Predicted: 6.887133
[0.5, 0.2973, 0.6667, 0.3692]
6.8871002

Real: 8.000000 	 Predicted: 9.531203
[0.5, 0.5946, 0.8333, 0.7077]
9.5309602

Real: 8.000000 	 Predicted: 6.539607
[0.0, 0.3784, 0.5, 0.4308]
6.5394108

Real: 8.000000 	 Predicted: 8.781522
[0.5, 0.2162, 0.8333, 0.3077]
8.7815882

Real: 7.000000 	 Predicted: 9.323570
[1.0, 0.973, 1.0, 0.9692]
9.3231762

Real: 9.000000 	 Predicted: 8.731816
[0.5, 0.8108, 0.8333, 0.8]
8.7314465



In [10]:
X_all

[[1.0, 1.0, 0.8333, 1.0],
 [0.5, 0.5405, 0.6667, 0.7231],
 [0.5, 0.4595, 0.8333, 0.7385],
 [0.5, 0.2973, 0.6667, 0.3692],
 [1.0, 0.973, 1.0, 0.9692],
 [1.0, 0.5405, 0.8333, 0.5538],
 [0.0, 0.1081, 0.5, 0.2462],
 [0.5, 0.8108, 0.8333, 0.8],
 [0.5, 0.5946, 0.8333, 0.7077],
 [0.0, 0.3784, 0.5, 0.4308],
 [0.5, 0.5405, 0.6667, 0.7231],
 [1.0, 0.973, 1.0, 0.9692],
 [0.0, 0.1351, 0.5, 0.2],
 [0.5, 0.4595, 0.8333, 0.7385],
 [0.5, 0.2973, 0.6667, 0.3692],
 [1.0, 1.0, 0.8333, 1.0],
 [0.5, 0.5946, 0.8333, 0.7077],
 [0.0, 0.2162, 0.5, 0.3385],
 [0.0, 0.3784, 0.5, 0.4308],
 [0.0, 0.1622, 0.3333, 0.2769],
 [0.0, 0.1351, 0.0, 0.2],
 [0.0, 0.1892, 0.6667, 0.2923],
 [0.5, 0.5405, 0.6667, 0.7231],
 [1.0, 1.0, 0.8333, 1.0],
 [1.0, 0.973, 1.0, 0.9692],
 [0.5, 0.4595, 0.8333, 0.7385],
 [0.0, 0.2162, 0.1667, 0.1846],
 [0.5, 0.2973, 0.6667, 0.3692],
 [0.5, 0.8108, 0.8333, 0.8],
 [1.0, 1.0, 0.8333, 1.0],
 [1.0, 0.5405, 0.8333, 0.5538],
 [0.5, 0.4595, 0.8333, 0.7385],
 [0.5, 0.5405, 0.6667, 0.7231],
 [0.5, 0.2

In [14]:
print "Features sorted by their score:"
#f = sorted(zip(map(lambda x: round(x, 4), regr.coef_), FEATURES), reverse=True)
f = zip(map(lambda x: round(x, 4), regr.coef_), FEATURES)
print regr.intercept_
print len(FEATURES)
for i in f:
    #print "%4f \t %s" %(i[0], i[1])
    print i[0]

Features sorted by their score:
0.0
4
-1.3192
-0.2916
1.6089
0.205


In [ ]:
ind = [map(lambda x: x[1], f).index(i) for i in FEATURES]
for i in ind:
    print i+1

In [ ]:
plt.plot([i[22] for i in X])
plt.show()

In [ ]:
print FEATURES[8]
[i[22] for i in X]

In [ ]:
for k in range(0,26):
    print FEATURES[k], 10 - [i[k] for i in X].count(0)